# WELCOME

This notebook will guide you through two increasingly significant applications in the realm of Generative AI: RAG (Retrieval Augmented Generation) chatbots and text summarization for big text.

Through two distinct projects, you will explore these technologies and enhance your skills. Detailed descriptions of the projects are provided below.

## Project 1: Building a Chatbot with a PDF Document (RAG)

In this project, you will develop a chatbot using a provided PDF document from web page. You will utilize the Langchain framework along with a large language model (LLM) such as GPT or Gemini. The chatbot will leverage the Retrieval Augmented Generation (RAG) technique to comprehend the document's content and respond to user queries effectively.

### **Project Steps:**

- **1.PDF Document Upload:** Upload the provided PDF document from web page (https://aclanthology.org/N19-1423.pdf) (BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding).

- **2.Chunking:** Divide the uploaded PDF document into smaller segments (chunks). This facilitates more efficient information processing by the LLM.

- **3.ChromaDB Setup:**
  - Save ChromaDB to your Google Drive.

  - Retrieve ChromaDB from your Drive to begin using it in your project.

  - ChromaDB serves as a vector database to store embedding vectors generated from your document.

- **4.Embedding Vectors Creation:**
  - Convert the chunked document into embedding vectors. You can use either GPT or Gemini embedding models for this purpose.

  - If you choose the Gemini embedding model, set "task_type" to "retrieval_document" when converting the chunked document.

- **5.Chatbot Development:**
  - Utilize the **load_qa_chain** function from the Langchain library to build the chatbot.

  - This function will interpret user queries, retrieve relevant information from **ChromaDB**, and generate responses accordingly.



### Install Libraries

In [1]:
 #inspect what exactly is going on inside your chain or agent. The best way to do this is with LangSmith.

import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

# https://smith.langchain.com/o/b249c21a-9672-4a8d-8cc6-c968dbdc2fef/projects/p/4e4bb92c-e1bf-4c57-b0a2-55976ce9d2e7?timeModel=%7B%22duration%22%3A%227d%22%7D&searchModel=%7B%22filter%22%3A%22%22%7D

··········


In [16]:
!pip install -qU langchain-community # access the tools and components (like DocumentLoaders, TextSplitters, VectorStores, and Retrievers)

!pip install -qU langchain-openai # utilize OpenAI's models for the "Generation" part

# !pip install -qU langchain-google-community        # Google's LLMs for the "Generation" part


!pip install -qU langchain-chroma # the "Store" step of the "Indexing" pipeline

!pip install -qU pypdfium2 #  the "Load" step of the "Indexing" pipeline, DocumentLoader to effectively extract text from PDF documents

### Access Google Drive

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Entering Your OpenAI or Google Gemini API Key.

In [18]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')

### Loading PDF Document

In [19]:

# PDF reader function
# It reads PDFs page by page and outputs Document objects (LangChain’s standard format). Returning a List[Document] → perfect, because the next step (chunking with RecursiveCharacterTextSplitter) expects this format
from langchain_community.document_loaders import PyPDFium2Loader

def read_doc(file_path: str):
    """
    Reads a PDF file and returns its pages as LangChain Document objects.
    Args:
        file_path (str): Path to the PDF file.
    Returns:
        List[Document]: A list of Document objects, one per page.
    """

    loader = PyPDFium2Loader(file_path)
    pdf_documents = loader.load()          # Reads PDF page by page
    return pdf_documents

In [20]:
pdf = read_doc('/content/drive/MyDrive/nlp/bert_article.pdf')

print(f"-" * 55)

print(f"The pdf file has {len(pdf)} pages.")



-------------------------------------------------------
The pdf file has 16 pages.


In [21]:
pdf

[Document(metadata={'producer': 'pdfTeX-1.40.18', 'creator': 'LaTeX with hyperref package', 'creationdate': '2019-04-29T17:36:03+00:00', 'title': 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding', 'author': 'Jacob Devlin ; Ming-Wei Chang ; Kenton Lee ; Kristina Toutanova', 'subject': 'N19-1 2019', 'keywords': '', 'moddate': '2019-04-29T17:36:03+00:00', 'source': '/content/drive/MyDrive/nlp/bert_article.pdf', 'total_pages': 16, 'page': 0}, page_content='Proceedings of NAACL-HLT 2019, pages 4171–4186\nMinneapolis, Minnesota, June 2 - June 7, 2019. \nc 2019 Association for Computational Linguistics\n4171\nBERT: Pre-training of Deep Bidirectional Transformers for\nLanguage Understanding\nJacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova\nGoogle AI Language\n{jacobdevlin,mingweichang,kentonl,kristout}@google.com\nAbstract\nWe introduce a new language representa\x02tion model called BERT, which stands for\nBidirectional Encoder Representations from\nT

In [22]:
pdf[0].page_content   # text of first page


'Proceedings of NAACL-HLT 2019, pages 4171–4186\nMinneapolis, Minnesota, June 2 - June 7, 2019. \nc 2019 Association for Computational Linguistics\n4171\nBERT: Pre-training of Deep Bidirectional Transformers for\nLanguage Understanding\nJacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova\nGoogle AI Language\n{jacobdevlin,mingweichang,kentonl,kristout}@google.com\nAbstract\nWe introduce a new language representa\x02tion model called BERT, which stands for\nBidirectional Encoder Representations from\nTransformers. Unlike recent language repre\x02sentation models (Peters et al., 2018a; Rad\x02ford et al., 2018), BERT is designed to pre\x02train deep bidirectional representations from\nunlabeled text by jointly conditioning on both\nleft and right context in all layers. As a re\x02sult, the pre-trained BERT model can be fine\x02tuned with just one additional output layer\nto create state-of-the-art models for a wide\nrange of tasks, such as question answering and\nlanguage inference,

In [23]:
print(pdf[0].page_content[:1250])

Proceedings of NAACL-HLT 2019, pages 4171–4186
Minneapolis, Minnesota, June 2 - June 7, 2019. 
c 2019 Association for Computational Linguistics
4171
BERT: Pre-training of Deep Bidirectional Transformers for
Language Understanding
Jacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova
Google AI Language
{jacobdevlin,mingweichang,kentonl,kristout}@google.com
Abstract
We introduce a new language representation model called BERT, which stands for
Bidirectional Encoder Representations from
Transformers. Unlike recent language representation models (Peters et al., 2018a; Radford et al., 2018), BERT is designed to pretrain deep bidirectional representations from
unlabeled text by jointly conditioning on both
left and right context in all layers. As a result, the pre-trained BERT model can be finetuned with just one additional output layer
to create state-of-the-art models for a wide
range of tasks, such as question answering and
language inference, without substantial taskspecific a

NameError: name 'Markdown' is not defined

In [28]:
from IPython.display import Markdown

Markdown(pdf[2].page_content)

4173
BERT BERT
E[CLS] E1
 E[SEP] ... EN
E1’ ... EM’
C T1 T[SEP] ... TN
T1’ ... TM’
[CLS] Tok 1 [SEP] ... Tok N Tok 1 ... TokM
Question Paragraph
Start/End Span
BERT
E[CLS] E1
 E[SEP] ... EN
E1’ ... EM’
C T1 T[SEP] ... TN
T1’ ... TM’
[CLS] Tok 1 [SEP] ... Tok N Tok 1 ... TokM
Masked Sentence A Masked Sentence B
Pre-training Fine-Tuning
NSP Mask LM Mask LM
Unlabeled Sentence A and B Pair 
SQuAD
Question Answer Pair
MNLI NER
Figure 1: Overall pre-training and fine-tuning procedures for BERT. Apart from output layers, the same architectures are used in both pre-training and fine-tuning. The same pre-trained model parameters are used to initialize
models for different down-stream tasks. During fine-tuning, all parameters are fine-tuned. [CLS] is a special
symbol added in front of every input example, and [SEP] is a special separator token (e.g. separating questions/answers).
ing and auto-encoder objectives have been used
for pre-training such models (Howard and Ruder,
2018; Radford et al., 2018; Dai and Le, 2015).
2.3 Transfer Learning from Supervised Data
There has also been work showing effective transfer from supervised tasks with large datasets, such
as natural language inference (Conneau et al.,
2017) and machine translation (McCann et al.,
2017). Computer vision research has also demonstrated the importance of transfer learning from
large pre-trained models, where an effective recipe
is to fine-tune models pre-trained with ImageNet (Deng et al., 2009; Yosinski et al., 2014).
3 BERT
We introduce BERT and its detailed implementation in this section. There are two steps in our
framework: pre-training and fine-tuning. During pre-training, the model is trained on unlabeled
data over different pre-training tasks. For finetuning, the BERT model is first initialized with
the pre-trained parameters, and all of the parameters are fine-tuned using labeled data from the
downstream tasks. Each downstream task has separate fine-tuned models, even though they are initialized with the same pre-trained parameters. The
question-answering example in Figure 1 will serve
as a running example for this section.
A distinctive feature of BERT is its unified architecture across different tasks. There is minimal difference between the pre-trained architecture and the final downstream architecture.
Model Architecture BERT’s model architecture is a multi-layer bidirectional Transformer encoder based on the original implementation described in Vaswani et al. (2017) and released in
the tensor2tensor library.1 Because the use
of Transformers has become common and our implementation is almost identical to the original,
we will omit an exhaustive background description of the model architecture and refer readers to
Vaswani et al. (2017) as well as excellent guides
such as “The Annotated Transformer.”2
In this work, we denote the number of layers
(i.e., Transformer blocks) as L, the hidden size as
H, and the number of self-attention heads as A.
3
We primarily report results on two model sizes:
BERTBASE (L=12, H=768, A=12, Total Parameters=110M) and BERTLARGE (L=24, H=1024,
A=16, Total Parameters=340M).
BERTBASE was chosen to have the same model
size as OpenAI GPT for comparison purposes.
Critically, however, the BERT Transformer uses
bidirectional self-attention, while the GPT Transformer uses constrained self-attention where every
token can only attend to context to its left.4
1
https://github.com/tensorflow/tensor2tensor
2
http://nlp.seas.harvard.edu/2018/04/03/attention.html
3
In all cases we set the feed-forward/filter size to be 4H,
i.e., 3072 for the H = 768 and 4096 for the H = 1024.
4We note that in the literature the bidirectional Trans-


In [29]:

# all pages







from IPython.display import Markdown, display

for i, d in enumerate(pdf, start=1):
    display(Markdown(f"### Page {i}\n\n{d.page_content}"))




### Page 1

Proceedings of NAACL-HLT 2019, pages 4171–4186
Minneapolis, Minnesota, June 2 - June 7, 2019. 
c 2019 Association for Computational Linguistics
4171
BERT: Pre-training of Deep Bidirectional Transformers for
Language Understanding
Jacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova
Google AI Language
{jacobdevlin,mingweichang,kentonl,kristout}@google.com
Abstract
We introduce a new language representation model called BERT, which stands for
Bidirectional Encoder Representations from
Transformers. Unlike recent language representation models (Peters et al., 2018a; Radford et al., 2018), BERT is designed to pretrain deep bidirectional representations from
unlabeled text by jointly conditioning on both
left and right context in all layers. As a result, the pre-trained BERT model can be finetuned with just one additional output layer
to create state-of-the-art models for a wide
range of tasks, such as question answering and
language inference, without substantial taskspecific architecture modifications.
BERT is conceptually simple and empirically
powerful. It obtains new state-of-the-art results on eleven natural language processing
tasks, including pushing the GLUE score to
80.5% (7.7% point absolute improvement),
MultiNLI accuracy to 86.7% (4.6% absolute
improvement), SQuAD v1.1 question answering Test F1 to 93.2 (1.5 point absolute improvement) and SQuAD v2.0 Test F1 to 83.1
(5.1 point absolute improvement).
1 Introduction
Language model pre-training has been shown to
be effective for improving many natural language
processing tasks (Dai and Le, 2015; Peters et al.,
2018a; Radford et al., 2018; Howard and Ruder,
2018). These include sentence-level tasks such as
natural language inference (Bowman et al., 2015;
Williams et al., 2018) and paraphrasing (Dolan
and Brockett, 2005), which aim to predict the relationships between sentences by analyzing them
holistically, as well as token-level tasks such as
named entity recognition and question answering,
where models are required to produce fine-grained
output at the token level (Tjong Kim Sang and
De Meulder, 2003; Rajpurkar et al., 2016).
There are two existing strategies for applying pre-trained language representations to downstream tasks: feature-based and fine-tuning. The
feature-based approach, such as ELMo (Peters
et al., 2018a), uses task-specific architectures that
include the pre-trained representations as additional features. The fine-tuning approach, such as
the Generative Pre-trained Transformer (OpenAI
GPT) (Radford et al., 2018), introduces minimal
task-specific parameters, and is trained on the
downstream tasks by simply fine-tuning all pretrained parameters. The two approaches share the
same objective function during pre-training, where
they use unidirectional language models to learn
general language representations.
We argue that current techniques restrict the
power of the pre-trained representations, especially for the fine-tuning approaches. The major limitation is that standard language models are
unidirectional, and this limits the choice of architectures that can be used during pre-training. For
example, in OpenAI GPT, the authors use a left-toright architecture, where every token can only attend to previous tokens in the self-attention layers
of the Transformer (Vaswani et al., 2017). Such restrictions are sub-optimal for sentence-level tasks,
and could be very harmful when applying finetuning based approaches to token-level tasks such
as question answering, where it is crucial to incorporate context from both directions.
In this paper, we improve the fine-tuning based
approaches by proposing BERT: Bidirectional
Encoder Representations from Transformers.
BERT alleviates the previously mentioned unidirectionality constraint by using a “masked language model” (MLM) pre-training objective, inspired by the Cloze task (Taylor, 1953). The
masked language model randomly masks some of
the tokens from the input, and the objective is to
predict the original vocabulary id of the masked


### Page 2

4172
word based only on its context. Unlike left-toright language model pre-training, the MLM objective enables the representation to fuse the left
and the right context, which allows us to pretrain a deep bidirectional Transformer. In addition to the masked language model, we also use
a “next sentence prediction” task that jointly pretrains text-pair representations. The contributions
of our paper are as follows:
• We demonstrate the importance of bidirectional
pre-training for language representations. Unlike Radford et al. (2018), which uses unidirectional language models for pre-training, BERT
uses masked language models to enable pretrained deep bidirectional representations. This
is also in contrast to Peters et al. (2018a), which
uses a shallow concatenation of independently
trained left-to-right and right-to-left LMs.
• We show that pre-trained representations reduce
the need for many heavily-engineered taskspecific architectures. BERT is the first finetuning based representation model that achieves
state-of-the-art performance on a large suite
of sentence-level and token-level tasks, outperforming many task-specific architectures.
• BERT advances the state of the art for eleven
NLP tasks. The code and pre-trained models are available at https://github.com/
google-research/bert.
2 Related Work
There is a long history of pre-training general language representations, and we briefly review the
most widely-used approaches in this section.
2.1 Unsupervised Feature-based Approaches
Learning widely applicable representations of
words has been an active area of research for
decades, including non-neural (Brown et al., 1992;
Ando and Zhang, 2005; Blitzer et al., 2006) and
neural (Mikolov et al., 2013; Pennington et al.,
2014) methods. Pre-trained word embeddings
are an integral part of modern NLP systems, offering significant improvements over embeddings
learned from scratch (Turian et al., 2010). To pretrain word embedding vectors, left-to-right language modeling objectives have been used (Mnih
and Hinton, 2009), as well as objectives to discriminate correct from incorrect words in left and
right context (Mikolov et al., 2013).
These approaches have been generalized to
coarser granularities, such as sentence embeddings (Kiros et al., 2015; Logeswaran and Lee,
2018) or paragraph embeddings (Le and Mikolov,
2014). To train sentence representations, prior
work has used objectives to rank candidate next
sentences (Jernite et al., 2017; Logeswaran and
Lee, 2018), left-to-right generation of next sentence words given a representation of the previous
sentence (Kiros et al., 2015), or denoising autoencoder derived objectives (Hill et al., 2016).
ELMo and its predecessor (Peters et al., 2017,
2018a) generalize traditional word embedding research along a different dimension. They extract
context-sensitive features from a left-to-right and a
right-to-left language model. The contextual representation of each token is the concatenation of
the left-to-right and right-to-left representations.
When integrating contextual word embeddings
with existing task-specific architectures, ELMo
advances the state of the art for several major NLP
benchmarks (Peters et al., 2018a) including question answering (Rajpurkar et al., 2016), sentiment
analysis (Socher et al., 2013), and named entity
recognition (Tjong Kim Sang and De Meulder,
2003). Melamud et al. (2016) proposed learning
contextual representations through a task to predict a single word from both left and right context
using LSTMs. Similar to ELMo, their model is
feature-based and not deeply bidirectional. Fedus
et al. (2018) shows that the cloze task can be used
to improve the robustness of text generation models.
2.2 Unsupervised Fine-tuning Approaches
As with the feature-based approaches, the first
works in this direction only pre-trained word embedding parameters from unlabeled text (Collobert and Weston, 2008).
More recently, sentence or document encoders
which produce contextual token representations
have been pre-trained from unlabeled text and
fine-tuned for a supervised downstream task (Dai
and Le, 2015; Howard and Ruder, 2018; Radford
et al., 2018). The advantage of these approaches
is that few parameters need to be learned from
scratch. At least partly due to this advantage,
OpenAI GPT (Radford et al., 2018) achieved previously state-of-the-art results on many sentencelevel tasks from the GLUE benchmark (Wang
et al., 2018a). Left-to-right language model-


### Page 3

4173
BERT BERT
E[CLS] E1
 E[SEP] ... EN
E1’ ... EM’
C T1 T[SEP] ... TN
T1’ ... TM’
[CLS] Tok 1 [SEP] ... Tok N Tok 1 ... TokM
Question Paragraph
Start/End Span
BERT
E[CLS] E1
 E[SEP] ... EN
E1’ ... EM’
C T1 T[SEP] ... TN
T1’ ... TM’
[CLS] Tok 1 [SEP] ... Tok N Tok 1 ... TokM
Masked Sentence A Masked Sentence B
Pre-training Fine-Tuning
NSP Mask LM Mask LM
Unlabeled Sentence A and B Pair 
SQuAD
Question Answer Pair
MNLI NER
Figure 1: Overall pre-training and fine-tuning procedures for BERT. Apart from output layers, the same architectures are used in both pre-training and fine-tuning. The same pre-trained model parameters are used to initialize
models for different down-stream tasks. During fine-tuning, all parameters are fine-tuned. [CLS] is a special
symbol added in front of every input example, and [SEP] is a special separator token (e.g. separating questions/answers).
ing and auto-encoder objectives have been used
for pre-training such models (Howard and Ruder,
2018; Radford et al., 2018; Dai and Le, 2015).
2.3 Transfer Learning from Supervised Data
There has also been work showing effective transfer from supervised tasks with large datasets, such
as natural language inference (Conneau et al.,
2017) and machine translation (McCann et al.,
2017). Computer vision research has also demonstrated the importance of transfer learning from
large pre-trained models, where an effective recipe
is to fine-tune models pre-trained with ImageNet (Deng et al., 2009; Yosinski et al., 2014).
3 BERT
We introduce BERT and its detailed implementation in this section. There are two steps in our
framework: pre-training and fine-tuning. During pre-training, the model is trained on unlabeled
data over different pre-training tasks. For finetuning, the BERT model is first initialized with
the pre-trained parameters, and all of the parameters are fine-tuned using labeled data from the
downstream tasks. Each downstream task has separate fine-tuned models, even though they are initialized with the same pre-trained parameters. The
question-answering example in Figure 1 will serve
as a running example for this section.
A distinctive feature of BERT is its unified architecture across different tasks. There is minimal difference between the pre-trained architecture and the final downstream architecture.
Model Architecture BERT’s model architecture is a multi-layer bidirectional Transformer encoder based on the original implementation described in Vaswani et al. (2017) and released in
the tensor2tensor library.1 Because the use
of Transformers has become common and our implementation is almost identical to the original,
we will omit an exhaustive background description of the model architecture and refer readers to
Vaswani et al. (2017) as well as excellent guides
such as “The Annotated Transformer.”2
In this work, we denote the number of layers
(i.e., Transformer blocks) as L, the hidden size as
H, and the number of self-attention heads as A.
3
We primarily report results on two model sizes:
BERTBASE (L=12, H=768, A=12, Total Parameters=110M) and BERTLARGE (L=24, H=1024,
A=16, Total Parameters=340M).
BERTBASE was chosen to have the same model
size as OpenAI GPT for comparison purposes.
Critically, however, the BERT Transformer uses
bidirectional self-attention, while the GPT Transformer uses constrained self-attention where every
token can only attend to context to its left.4
1
https://github.com/tensorflow/tensor2tensor
2
http://nlp.seas.harvard.edu/2018/04/03/attention.html
3
In all cases we set the feed-forward/filter size to be 4H,
i.e., 3072 for the H = 768 and 4096 for the H = 1024.
4We note that in the literature the bidirectional Trans-


### Page 4

4174
Input/Output Representations To make BERT
handle a variety of down-stream tasks, our input
representation is able to unambiguously represent
both a single sentence and a pair of sentences
(e.g., h Question, Answeri) in one token sequence.
Throughout this work, a “sentence” can be an arbitrary span of contiguous text, rather than an actual
linguistic sentence. A “sequence” refers to the input token sequence to BERT, which may be a single sentence or two sentences packed together.
We use WordPiece embeddings (Wu et al.,
2016) with a 30,000 token vocabulary. The first
token of every sequence is always a special classification token ([CLS]). The final hidden state
corresponding to this token is used as the aggregate sequence representation for classification
tasks. Sentence pairs are packed together into a
single sequence. We differentiate the sentences in
two ways. First, we separate them with a special
token ([SEP]). Second, we add a learned embedding to every token indicating whether it belongs
to sentence A or sentence B. As shown in Figure 1,
we denote input embedding as E, the final hidden
vector of the special [CLS] token as C ∈ R
H,
and the final hidden vector for the i
th input token
as Ti ∈ R
H.
For a given token, its input representation is
constructed by summing the corresponding token,
segment, and position embeddings. A visualization of this construction can be seen in Figure 2.
3.1 Pre-training BERT
Unlike Peters et al. (2018a) and Radford et al.
(2018), we do not use traditional left-to-right or
right-to-left language models to pre-train BERT.
Instead, we pre-train BERT using two unsupervised tasks, described in this section. This step
is presented in the left part of Figure 1.
Task #1: Masked LM Intuitively, it is reasonable to believe that a deep bidirectional model is
strictly more powerful than either a left-to-right
model or the shallow concatenation of a left-toright and a right-to-left model. Unfortunately,
standard conditional language models can only be
trained left-to-right or right-to-left, since bidirectional conditioning would allow each word to indirectly “see itself”, and the model could trivially
predict the target word in a multi-layered context.
former is often referred to as a “Transformer encoder” while
the left-context-only version is referred to as a “Transformer
decoder” since it can be used for text generation.
In order to train a deep bidirectional representation, we simply mask some percentage of the input
tokens at random, and then predict those masked
tokens. We refer to this procedure as a “masked
LM” (MLM), although it is often referred to as a
Cloze task in the literature (Taylor, 1953). In this
case, the final hidden vectors corresponding to the
mask tokens are fed into an output softmax over
the vocabulary, as in a standard LM. In all of our
experiments, we mask 15% of all WordPiece tokens in each sequence at random. In contrast to
denoising auto-encoders (Vincent et al., 2008), we
only predict the masked words rather than reconstructing the entire input.
Although this allows us to obtain a bidirectional pre-trained model, a downside is that we
are creating a mismatch between pre-training and
fine-tuning, since the [MASK] token does not appear during fine-tuning. To mitigate this, we do
not always replace “masked” words with the actual [MASK] token. The training data generator
chooses 15% of the token positions at random for
prediction. If the i-th token is chosen, we replace
the i-th token with (1) the [MASK] token 80% of
the time (2) a random token 10% of the time (3)
the unchanged i-th token 10% of the time. Then,
Ti will be used to predict the original token with
cross entropy loss. We compare variations of this
procedure in Appendix C.2.
Task #2: Next Sentence Prediction (NSP)
Many important downstream tasks such as Question Answering (QA) and Natural Language Inference (NLI) are based on understanding the relationship between two sentences, which is not directly captured by language modeling. In order
to train a model that understands sentence relationships, we pre-train for a binarized next sentence prediction task that can be trivially generated from any monolingual corpus. Specifically,
when choosing the sentences A and B for each pretraining example, 50% of the time B is the actual
next sentence that follows A (labeled as IsNext),
and 50% of the time it is a random sentence from
the corpus (labeled as NotNext). As we show
in Figure 1, C is used for next sentence prediction (NSP).5 Despite its simplicity, we demonstrate in Section 5.1 that pre-training towards this
task is very beneficial to both QA and NLI. 6
5The final model achieves 97%-98% accuracy on NSP.
6The vector C is not a meaningful sentence representation
without fine-tuning, since it was trained with NSP.


### Page 5

4175
[CLS] my dog is cute [SEP] he likes play ##ing [SEP] Input
E[CLS] Ehe Elikes Eplay E##ing E[SEP] Emy Edog Eis Ecute E[SEP]
Token
Embeddings
EAEBEBEBEBEBEAEAEAEAEA
Segment
Embeddings
E0 E6E7E8E9E10 E1E2E3E4E5
Position
Embeddings
Figure 2: BERT input representation. The input embeddings are the sum of the token embeddings, the segmentation embeddings and the position embeddings.
The NSP task is closely related to representationlearning objectives used in Jernite et al. (2017) and
Logeswaran and Lee (2018). However, in prior
work, only sentence embeddings are transferred to
down-stream tasks, where BERT transfers all parameters to initialize end-task model parameters.
Pre-training data The pre-training procedure
largely follows the existing literature on language
model pre-training. For the pre-training corpus we
use the BooksCorpus (800M words) (Zhu et al.,
2015) and English Wikipedia (2,500M words).
For Wikipedia we extract only the text passages
and ignore lists, tables, and headers. It is critical to use a document-level corpus rather than a
shuffled sentence-level corpus such as the Billion
Word Benchmark (Chelba et al., 2013) in order to
extract long contiguous sequences.
3.2 Fine-tuning BERT
Fine-tuning is straightforward since the selfattention mechanism in the Transformer allows BERT to model many downstream tasks—
whether they involve single text or text pairs—by
swapping out the appropriate inputs and outputs.
For applications involving text pairs, a common
pattern is to independently encode text pairs before applying bidirectional cross attention, such
as Parikh et al. (2016); Seo et al. (2017). BERT
instead uses the self-attention mechanism to unify
these two stages, as encoding a concatenated text
pair with self-attention effectively includes bidirectional cross attention between two sentences.
For each task, we simply plug in the taskspecific inputs and outputs into BERT and finetune all the parameters end-to-end. At the input, sentence A and sentence B from pre-training
are analogous to (1) sentence pairs in paraphrasing, (2) hypothesis-premise pairs in entailment, (3)
question-passage pairs in question answering, and
(4) a degenerate text-∅ pair in text classification
or sequence tagging. At the output, the token representations are fed into an output layer for tokenlevel tasks, such as sequence tagging or question
answering, and the [CLS] representation is fed
into an output layer for classification, such as entailment or sentiment analysis.
Compared to pre-training, fine-tuning is relatively inexpensive. All of the results in the paper can be replicated in at most 1 hour on a single Cloud TPU, or a few hours on a GPU, starting
from the exact same pre-trained model.7 We describe the task-specific details in the corresponding subsections of Section 4. More details can be
found in Appendix A.5.
4 Experiments
In this section, we present BERT fine-tuning results on 11 NLP tasks.
4.1 GLUE
The General Language Understanding Evaluation
(GLUE) benchmark (Wang et al., 2018a) is a collection of diverse natural language understanding
tasks. Detailed descriptions of GLUE datasets are
included in Appendix B.1.
To fine-tune on GLUE, we represent the input
sequence (for single sentence or sentence pairs)
as described in Section 3, and use the final hidden vector C ∈ R
H corresponding to the first
input token ([CLS]) as the aggregate representation. The only new parameters introduced during
fine-tuning are classification layer weights W ∈
R
K×H, where K is the number of labels. We compute a standard classification loss with C and W,
i.e., log(softmax(CWT)).
7
For example, the BERT SQuAD model can be trained in
around 30 minutes on a single Cloud TPU to achieve a Dev
F1 score of 91.0%.
8
See (10) in https://gluebenchmark.com/faq.


...

### Page 12

4182
for natural language understanding. In Proceedings
of the 2018 EMNLP Workshop BlackboxNLP: Analyzing and Interpreting Neural Networks for NLP,
pages 353–355.
Wei Wang, Ming Yan, and Chen Wu. 2018b. Multigranularity hierarchical attention fusion networks
for reading comprehension and question answering.
In Proceedings of the 56th Annual Meeting of the Association for Computational Linguistics (Volume 1:
Long Papers). Association for Computational Linguistics.
Alex Warstadt, Amanpreet Singh, and Samuel R Bowman. 2018. Neural network acceptability judgments. arXiv preprint arXiv:1805.12471.
Adina Williams, Nikita Nangia, and Samuel R Bowman. 2018. A broad-coverage challenge corpus
for sentence understanding through inference. In
NAACL.
Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc V
Le, Mohammad Norouzi, Wolfgang Macherey,
Maxim Krikun, Yuan Cao, Qin Gao, Klaus
Macherey, et al. 2016. Google’s neural machine translation system: Bridging the gap between
human and machine translation. arXiv preprint
arXiv:1609.08144.
Jason Yosinski, Jeff Clune, Yoshua Bengio, and Hod
Lipson. 2014. How transferable are features in deep
neural networks? In Advances in neural information
processing systems, pages 3320–3328.
Adams Wei Yu, David Dohan, Minh-Thang Luong, Rui
Zhao, Kai Chen, Mohammad Norouzi, and Quoc V
Le. 2018. QANet: Combining local convolution
with global self-attention for reading comprehension. In ICLR.
Rowan Zellers, Yonatan Bisk, Roy Schwartz, and Yejin
Choi. 2018. Swag: A large-scale adversarial dataset
for grounded commonsense inference. In Proceedings of the 2018 Conference on Empirical Methods
in Natural Language Processing (EMNLP).
Yukun Zhu, Ryan Kiros, Rich Zemel, Ruslan Salakhutdinov, Raquel Urtasun, Antonio Torralba, and Sanja
Fidler. 2015. Aligning books and movies: Towards
story-like visual explanations by watching movies
and reading books. In Proceedings of the IEEE
international conference on computer vision, pages
19–27.
Appendix for “BERT: Pre-training of
Deep Bidirectional Transformers for
Language Understanding”
We organize the appendix into three sections:
• Additional implementation details for BERT
are presented in Appendix A;
• Additional details for our experiments are
presented in Appendix B; and
• Additional ablation studies are presented in
Appendix C.
We present additional ablation studies for
BERT including:
– Effect of Number of Training Steps; and
– Ablation for Different Masking Procedures.
A Additional Details for BERT
A.1 Illustration of the Pre-training Tasks
We provide examples of the pre-training tasks in
the following.
Masked LM and the Masking Procedure Assuming the unlabeled sentence is my dog is
hairy, and during the random masking procedure
we chose the 4-th token (which corresponding to
hairy), our masking procedure can be further illustrated by
• 80% of the time: Replace the word with the
[MASK] token, e.g., my dog is hairy →
my dog is [MASK]
• 10% of the time: Replace the word with a
random word, e.g., my dog is hairy → my
dog is apple
• 10% of the time: Keep the word unchanged, e.g., my dog is hairy → my dog
is hairy. The purpose of this is to bias the
representation towards the actual observed
word.
The advantage of this procedure is that the
Transformer encoder does not know which words
it will be asked to predict or which have been replaced by random words, so it is forced to keep
a distributional contextual representation of every input token. Additionally, because random
replacement only occurs for 1.5% of all tokens
(i.e., 10% of 15%), this does not seem to harm
the model’s language understanding capability. In
Section C.2, we evaluate the impact this procedure.
Compared to standard langauge model training,
the masked LM only make predictions on 15% of
tokens in each batch, which suggests that more
pre-training steps may be required for the model


### Page 13

4183
BERT (Ours)
Trm Trm Trm
Trm Trm Trm
...
...
Trm Trm Trm
Trm Trm Trm
...
...
OpenAI GPT
Lstm
ELMo
Lstm Lstm
Lstm Lstm Lstm
Lstm Lstm Lstm
Lstm Lstm Lstm
 T1 T2
 TN
...
...
...
...
...
 E1 E2
 EN
...
 T1 T2 TN
...
 E1 E2
 E ... N
 T1 T2
 TN
...
 E1 E2
 EN
...
Figure 3: Differences in pre-training model architectures. BERT uses a bidirectional Transformer. OpenAI GPT
uses a left-to-right Transformer. ELMo uses the concatenation of independently trained left-to-right and right-toleft LSTMs to generate features for downstream tasks. Among the three, only BERT representations are jointly
conditioned on both left and right context in all layers. In addition to the architecture differences, BERT and
OpenAI GPT are fine-tuning approaches, while ELMo is a feature-based approach.
to converge. In Section C.1 we demonstrate that
MLM does converge marginally slower than a leftto-right model (which predicts every token), but
the empirical improvements of the MLM model
far outweigh the increased training cost.
Next Sentence Prediction The next sentence
prediction task can be illustrated in the following
examples.
Input = [CLS] the man went to [MASK] store [SEP]
he bought a gallon [MASK] milk [SEP]
Label = IsNext
Input = [CLS] the man [MASK] to the store [SEP]
penguin [MASK] are flight ##less birds [SEP]
Label = NotNext
A.2 Pre-training Procedure
To generate each training input sequence, we sample two spans of text from the corpus, which we
refer to as “sentences” even though they are typically much longer than single sentences (but can
be shorter also). The first sentence receives the A
embedding and the second receives the B embedding. 50% of the time B is the actual next sentence
that follows A and 50% of the time it is a random
sentence, which is done for the “next sentence prediction” task. They are sampled such that the combined length is ≤ 512 tokens. The LM masking is
applied after WordPiece tokenization with a uniform masking rate of 15%, and no special consideration given to partial word pieces.
We train with batch size of 256 sequences (256
sequences * 512 tokens = 128,000 tokens/batch)
for 1,000,000 steps, which is approximately 40
epochs over the 3.3 billion word corpus. We
use Adam with learning rate of 1e-4, β1 = 0.9,
β2 = 0.999, L2 weight decay of 0.01, learning
rate warmup over the first 10,000 steps, and linear
decay of the learning rate. We use a dropout probability of 0.1 on all layers. We use a gelu activation (Hendrycks and Gimpel, 2016) rather than
the standard relu, following OpenAI GPT. The
training loss is the sum of the mean masked LM
likelihood and the mean next sentence prediction
likelihood.
Training of BERTBASE was performed on 4
Cloud TPUs in Pod configuration (16 TPU chips
total).13 Training of BERTLARGE was performed
on 16 Cloud TPUs (64 TPU chips total). Each pretraining took 4 days to complete.
Longer sequences are disproportionately expensive because attention is quadratic to the sequence
length. To speed up pretraing in our experiments,
we pre-train the model with sequence length of
128 for 90% of the steps. Then, we train the rest
10% of the steps of sequence of 512 to learn the
positional embeddings.
A.3 Fine-tuning Procedure
For fine-tuning, most model hyperparameters are
the same as in pre-training, with the exception of
the batch size, learning rate, and number of training epochs. The dropout probability was always
kept at 0.1. The optimal hyperparameter values
are task-specific, but we found the following range
of possible values to work well across all tasks:
• Batch size: 16, 32
13https://cloudplatform.googleblog.com/2018/06/CloudTPU-now-offers-preemptible-pricing-and-globalavailability.html


### Page 14

4184
• Learning rate (Adam): 5e-5, 3e-5, 2e-5
• Number of epochs: 2, 3, 4
We also observed that large data sets (e.g.,
100k+ labeled training examples) were far less
sensitive to hyperparameter choice than small data
sets. Fine-tuning is typically very fast, so it is reasonable to simply run an exhaustive search over
the above parameters and choose the model that
performs best on the development set.
A.4 Comparison of BERT, ELMo ,and
OpenAI GPT
Here we studies the differences in recent popular
representation learning models including ELMo,
OpenAI GPT and BERT. The comparisons between the model architectures are shown visually
in Figure 3. Note that in addition to the architecture differences, BERT and OpenAI GPT are finetuning approaches, while ELMo is a feature-based
approach.
The most comparable existing pre-training
method to BERT is OpenAI GPT, which trains a
left-to-right Transformer LM on a large text corpus. In fact, many of the design decisions in BERT
were intentionally made to make it as close to
GPT as possible so that the two methods could be
minimally compared. The core argument of this
work is that the bi-directionality and the two pretraining tasks presented in Section 3.1 account for
the majority of the empirical improvements, but
we do note that there are several other differences
between how BERT and GPT were trained:
• GPT is trained on the BooksCorpus (800M
words); BERT is trained on the BooksCorpus (800M words) and Wikipedia (2,500M
words).
• GPT uses a sentence separator ([SEP]) and
classifier token ([CLS]) which are only introduced at fine-tuning time; BERT learns
[SEP], [CLS] and sentence A/B embeddings during pre-training.
• GPT was trained for 1M steps with a batch
size of 32,000 words; BERT was trained for
1M steps with a batch size of 128,000 words.
• GPT used the same learning rate of 5e-5 for
all fine-tuning experiments; BERT chooses a
task-specific fine-tuning learning rate which
performs the best on the development set.
To isolate the effect of these differences, we perform ablation experiments in Section 5.1 which
demonstrate that the majority of the improvements
are in fact coming from the two pre-training tasks
and the bidirectionality they enable.
A.5 Illustrations of Fine-tuning on Different
Tasks
The illustration of fine-tuning BERT on different
tasks can be seen in Figure 4. Our task-specific
models are formed by incorporating BERT with
one additional output layer, so a minimal number of parameters need to be learned from scratch.
Among the tasks, (a) and (b) are sequence-level
tasks while (c) and (d) are token-level tasks. In
the figure, E represents the input embedding, Ti
represents the contextual representation of token i,
[CLS] is the special symbol for classification output, and [SEP] is the special symbol to separate
non-consecutive token sequences.
B Detailed Experimental Setup
B.1 Detailed Descriptions for the GLUE
Benchmark Experiments.
The GLUE benchmark includes the following
datasets, the descriptions of which were originally
summarized in Wang et al. (2018a):
MNLI Multi-Genre Natural Language Inference
is a large-scale, crowdsourced entailment classification task (Williams et al., 2018). Given a pair of
sentences, the goal is to predict whether the second sentence is an entailment, contradiction, or
neutral with respect to the first one.
QQP Quora Question Pairs is a binary classification task where the goal is to determine if two
questions asked on Quora are semantically equivalent (Chen et al., 2018).
QNLI Question Natural Language Inference is
a version of the Stanford Question Answering
Dataset (Rajpurkar et al., 2016) which has been
converted to a binary classification task (Wang
et al., 2018a). The positive examples are (question, sentence) pairs which do contain the correct
answer, and the negative examples are (question,
sentence) from the same paragraph which do not
contain the answer.
SST-2 The Stanford Sentiment Treebank is a
binary single-sentence classification task consisting of sentences extracted from movie reviews


### Page 15

4185
BERT
E[CLS] E1
 E[SEP] ... EN
E1’ ... EM’
C T1 T[SEP] ... TN
T1’ ... TM’
[CLS] Tok 
1
... [SEP] Tok 
N
Tok 
1
... Tok
M
Question Paragraph
BERT
E[CLS] E1 E2
 EN
C T1 T2
 TN
Single Sentence 
...
...
BERT
[CLS] Tok 1 Tok 2 ... Tok N
E[CLS] E1 E2
 EN
C T1 T2
 TN
Single Sentence 
O B-PER O
...
E ... [CLS] E1
 E[SEP]
Class 
Label
... EN
E1’ ... EM’
C T1 T[SEP] ... TN
T1’ ... TM’
Start/End Span
Class 
Label
BERT
[CLS] Tok [CLS] Tok 1 Tok 2 ... Tok N
1
... [SEP] Tok 
N
Tok 
1
... Tok
M
Sentence 1
...
Sentence 2
Figure 4: Illustrations of Fine-tuning BERT on Different Tasks.
with human annotations of their sentiment (Socher
et al., 2013).
CoLA The Corpus of Linguistic Acceptability is
a binary single-sentence classification task, where
the goal is to predict whether an English sentence
is linguistically “acceptable” or not (Warstadt
et al., 2018).
STS-B The Semantic Textual Similarity Benchmark is a collection of sentence pairs drawn from
news headlines and other sources (Cer et al.,
2017). They were annotated with a score from 1
to 5 denoting how similar the two sentences are in
terms of semantic meaning.
MRPC Microsoft Research Paraphrase Corpus
consists of sentence pairs automatically extracted
from online news sources, with human annotations
for whether the sentences in the pair are semantically equivalent (Dolan and Brockett, 2005).
RTE Recognizing Textual Entailment is a binary entailment task similar to MNLI, but with
much less training data (Bentivogli et al., 2009).14
WNLI Winograd NLI is a small natural language inference dataset (Levesque et al., 2011).
The GLUE webpage notes that there are issues
with the construction of this dataset, 15 and every
trained system that’s been submitted to GLUE has
performed worse than the 65.1 baseline accuracy
of predicting the majority class. We therefore exclude this set to be fair to OpenAI GPT. For our
GLUE submission, we always predicted the majority class.
14Note that we only report single-task fine-tuning results
in this paper. A multitask fine-tuning approach could potentially push the performance even further. For example, we
did observe substantial improvements on RTE from multitask training with MNLI.
15https://gluebenchmark.com/faq


### Page 16

4186
C Additional Ablation Studies
C.1 Effect of Number of Training Steps
Figure 5 presents MNLI Dev accuracy after finetuning from a checkpoint that has been pre-trained
for k steps. This allows us to answer the following
questions:
1. Question: Does BERT really need such
a large amount of pre-training (128,000
words/batch * 1,000,000 steps) to achieve
high fine-tuning accuracy?
Answer: Yes, BERTBASE achieves almost
1.0% additional accuracy on MNLI when
trained on 1M steps compared to 500k steps.
2. Question: Does MLM pre-training converge
slower than LTR pre-training, since only 15%
of words are predicted in each batch rather
than every word?
Answer: The MLM model does converge
slightly slower than the LTR model. However, in terms of absolute accuracy the MLM
model begins to outperform the LTR model
almost immediately.
C.2 Ablation for Different Masking
Procedures
In Section 3.1, we mention that BERT uses a
mixed strategy for masking the target tokens when
pre-training with the masked language model
(MLM) objective. The following is an ablation
study to evaluate the effect of different masking
strategies.
Note that the purpose of the masking strategies
is to reduce the mismatch between pre-training
200 400 600 800 1,000
76
78
80
82
84
Pre-training Steps (Thousands)
MNLI Dev Accuracy
BERTBASE (Masked LM)
BERTBASE (Left-to-Right)
Figure 5: Ablation over number of training steps. This
shows the MNLI accuracy after fine-tuning, starting
from model parameters that have been pre-trained for
k steps. The x-axis is the value of k.
and fine-tuning, as the [MASK] symbol never appears during the fine-tuning stage. We report the
Dev results for both MNLI and NER. For NER,
we report both fine-tuning and feature-based approaches, as we expect the mismatch will be amplified for the feature-based approach as the model
will not have the chance to adjust the representations.
Masking Rates Dev Set Results
MASK SAME RND MNLI NER
Fine-tune Fine-tune Feature-based
80% 10% 10% 84.2 95.4 94.9
100% 0% 0% 84.3 94.9 94.0
80% 0% 20% 84.1 95.2 94.6
80% 20% 0% 84.4 95.2 94.7
0% 20% 80% 83.7 94.8 94.6
0% 0% 100% 83.6 94.9 94.6
Table 8: Ablation over different masking strategies.
The results are presented in Table 8. In the table,
MASK means that we replace the target token with
the [MASK] symbol for MLM; SAME means that
we keep the target token as is; RND means that
we replace the target token with another random
token.
The numbers in the left part of the table represent the probabilities of the specific strategies used
during MLM pre-training (BERT uses 80%, 10%,
10%). The right part of the paper represents the
Dev set results. For the feature-based approach,
we concatenate the last 4 layers of BERT as the
features, which was shown to be the best approach
in Section 5.3.
From the table it can be seen that fine-tuning is
surprisingly robust to different masking strategies.
However, as expected, using only the MASK strategy was problematic when applying the featurebased approach to NER. Interestingly, using only
the RND strategy performs much worse than our
strategy as well.


### Document Splitter

In [30]:
from langchain.text_splitter import RecursiveCharacterTextSplitter #CharacterTextSplitter


def chunk_data(docs, chunk_size=1000, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                                   chunk_overlap=chunk_overlap)
    # each chunk will be a maximum of 1000 characters long# each chunk will share a 200-character overlap with the previous chunk to maintain context
    pdf = text_splitter.split_documents(docs)
    return pdf



# This code splits documents into chunks using the RecursiveCharacterTextSplitter class from the langchain library.

# A function named chunk_data is defined, which takes a document or a collection of documents (docs) as input. It also takes two parameters:
# chunk_size and chunk_overlap. chunk_size specifies the maximum number of characters in each chunk, while chunk_overlap determines the amount of
# overlap between consecutive chunks.

# The function divides the documents into chunks based on these parameters using the RecursiveCharacterTextSplitter class. Consequently, each chunk contains chunk_size characters, with an overlap of chunk_overlap characters between consecutive chunks.

# As a result, the documents are segmented into chunks of specified sizes, and these chunks are returned.

# The chunk_overlap parameter is used to specify the sharing of characters between consecutive chunks. In other words, it ensures that the characters at
# the end of one chunk reappear at the beginning of the next chunk. This prevents the loss of information when the text is segmented or divided and
# helps preserve a certain context. Especially, overlap can be used to maintain important contextual relationships within a specific text and sustain meaning across chunks.

In [31]:
pdf_doc = chunk_data(docs=pdf)

len(pdf_doc)

83

In [32]:
pdf_doc[0:2]

[Document(metadata={'producer': 'pdfTeX-1.40.18', 'creator': 'LaTeX with hyperref package', 'creationdate': '2019-04-29T17:36:03+00:00', 'title': 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding', 'author': 'Jacob Devlin ; Ming-Wei Chang ; Kenton Lee ; Kristina Toutanova', 'subject': 'N19-1 2019', 'keywords': '', 'moddate': '2019-04-29T17:36:03+00:00', 'source': '/content/drive/MyDrive/nlp/bert_article.pdf', 'total_pages': 16, 'page': 0}, page_content='Proceedings of NAACL-HLT 2019, pages 4171–4186\nMinneapolis, Minnesota, June 2 - June 7, 2019. \nc 2019 Association for Computational Linguistics\n4171\nBERT: Pre-training of Deep Bidirectional Transformers for\nLanguage Understanding\nJacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova\nGoogle AI Language\n{jacobdevlin,mingweichang,kentonl,kristout}@google.com\nAbstract\nWe introduce a new language representa\x02tion model called BERT, which stands for\nBidirectional Encoder Representations from\nT

In [33]:
Markdown(pdf_doc[82].page_content)

MASK means that we replace the target token with
the [MASK] symbol for MLM; SAME means that
we keep the target token as is; RND means that
we replace the target token with another random
token.
The numbers in the left part of the table represent the probabilities of the specific strategies used
during MLM pre-training (BERT uses 80%, 10%,
10%). The right part of the paper represents the
Dev set results. For the feature-based approach,
we concatenate the last 4 layers of BERT as the
features, which was shown to be the best approach
in Section 5.3.
From the table it can be seen that fine-tuning is
surprisingly robust to different masking strategies.
However, as expected, using only the MASK strategy was problematic when applying the featurebased approach to NER. Interestingly, using only
the RND strategy performs much worse than our
strategy as well.

### 1. Creating A Embedding Model
### 2. Convert the Each Chunk of The Split Document to Embedding Vectors
### 3. Storing of The Embedding Vectors to Vectorstore
### 4. Save the Vectorstore to Your Drive

In [34]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large",
                              dimensions=3072)  # dimensions=256, 1024, 3072
embeddings

# As the embedding model, we use Openai's latest introduced text-embedding-3-large model.
# dimensions of text-embedding-3-large are 256, 1024 and 3072

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7e260d54f800>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7e260908dfd0>, model='text-embedding-3-large', dimensions=3072, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [35]:
# the "Embed" step of the RAG (Retrieval Augmented Generation) pipeline.
# This is the third stage in the indexing process, after "Load" and "Split".

In [36]:
# store the embedding vectors.

In [37]:
!pip install -qU langchain-chroma
from langchain_chroma import Chroma


# VectorStore object
# Embeds each Document in pdf_doc using the embeddings function. Stores vectors + metadata in a Chroma vector database
index = Chroma.from_documents(documents=pdf_doc,
                              embedding=embeddings,
                              persist_directory="./chroma_store_bert_z")  # persist_directory, saves in the directory

# Create a retriever object from our Chroma vector store (index). Wraps the vector store as a Retriever that returns the k most relevant chunks for a query.
retriever = index.as_retriever()  # for using agents #index.as_retriever(search_kwargs={"k": 4})



In [38]:
# refinement of the "Retrieval" step in a RAG system.
# to retrieve the four most relevant pieces of information from the indexed PDF

retriever = index.as_retriever(search_kwargs={"k": 4})

#
#


In [39]:
retriever.invoke("Describe BERT model architecture")

[Document(id='10ffc67d-9f78-408d-9d50-9acf62d59031', metadata={'subject': 'N19-1 2019', 'producer': 'pdfTeX-1.40.18', 'creationdate': '2019-04-29T17:36:03+00:00', 'page': 2, 'author': 'Jacob Devlin ; Ming-Wei Chang ; Kenton Lee ; Kristina Toutanova', 'source': '/content/drive/MyDrive/nlp/bert_article.pdf', 'keywords': '', 'moddate': '2019-04-29T17:36:03+00:00', 'total_pages': 16, 'creator': 'LaTeX with hyperref package', 'title': 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding'}, page_content='framework: pre-training and fine-tuning. Dur\x02ing pre-training, the model is trained on unlabeled\ndata over different pre-training tasks. For fine\x02tuning, the BERT model is first initialized with\nthe pre-trained parameters, and all of the param\x02eters are fine-tuned using labeled data from the\ndownstream tasks. Each downstream task has sep\x02arate fine-tuned models, even though they are ini\x02tialized with the same pre-trained parameters. The\nquestio

### Load Vectorstore(index) From Your Drive

In [40]:
# Initiates the Chroma class, which serves as our vector database.

loaded_from_drive_index = Chroma(persist_directory="/content/drive/MyDrive/nlp/bert_article_chroma_store",
                      embedding_function=embeddings)
#/chroma_store_bert_z

In [41]:
load_retriver = loaded_from_drive_index.as_retriever(search_kwargs={"k": 5})
load_retriver

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7e260cfa34d0>, search_kwargs={'k': 5})

In [42]:
# bonus ... yukardakilerin kisa hali .







def retrieve_query(query, k=4):
    retriever = index.as_retriever(search_kwargs={"k": k})  # loaded_index
    return retriever.invoke(query)

# The query we ask is first converted into an embedding vector. Then, using the cosine similarity metric, the similarity scores between this vector and the embedding vectors in the vector store are calculated and sorted. The top 4 most similar texts are returned

In [43]:
our_query = "Describe BERT model architecture"

doc_search = retrieve_query(our_query, k=5)  # first 5 most similar texts are returned
doc_search

[Document(id='10ffc67d-9f78-408d-9d50-9acf62d59031', metadata={'keywords': '', 'creator': 'LaTeX with hyperref package', 'moddate': '2019-04-29T17:36:03+00:00', 'subject': 'N19-1 2019', 'total_pages': 16, 'creationdate': '2019-04-29T17:36:03+00:00', 'producer': 'pdfTeX-1.40.18', 'page': 2, 'source': '/content/drive/MyDrive/nlp/bert_article.pdf', 'author': 'Jacob Devlin ; Ming-Wei Chang ; Kenton Lee ; Kristina Toutanova', 'title': 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding'}, page_content='framework: pre-training and fine-tuning. Dur\x02ing pre-training, the model is trained on unlabeled\ndata over different pre-training tasks. For fine\x02tuning, the BERT model is first initialized with\nthe pre-trained parameters, and all of the param\x02eters are fine-tuned using labeled data from the\ndownstream tasks. Each downstream task has sep\x02arate fine-tuned models, even though they are ini\x02tialized with the same pre-trained parameters. The\nquestio

In [44]:






# for now we only saw our pdf relevant parts, llm was not involved in ...








### Generating an Answer Based on The Similar Chunks

In [45]:
# vital bridge between the "Retrieval" and "Generation" phases of our RAG pipeline
# relevant information retrieved from our vector store (doc_search) and formats it, along with the user's question, into a clean and structured prompt that can be understood and used by an LLM to generate a final, coherent answer.

### Pipeline For RAG

In [60]:
  # 1. chatbot qa yapisi prompt kismi


from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.question_answering import load_qa_chain
from IPython.display import Markdown

# Role-based chat prompt
chat_template = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a scientific QA assistant. Use only the provided context. "
        "Always cite page numbers. If uncertain, say 'I don't know.' "
        "Keep answers short, neutral, and evidence-based."
    ),
    (
        "system",
        "Return output as a markdown block with sections:\n"
        "# Answer\n"
        "## Key Points (with page cites)\n"
        "### Sources\n"
        "### Confidence"
    ),
    (
        "human",
        # We inject both the raw context and a compact citations map the model can quote.
        "Context:\n{context}\n\n"
        "Citations Map (use these for page refs):\n{citations}\n\n"
        "Question: {question}"
    )
])

# For map_reduce you also need a combine prompt (roles preserved)
combine_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a scientific QA assistant. Combine the partial answers below into a single, consistent answer. "
     "Resolve conflicts conservatively; if evidence is insufficient, say 'I don't know.' "
     "Keep the tone neutral and cite page numbers where available."),
    ("system",
     "Return markdown with sections:\n"
     "### Answer\n"
     "### Key Points (with page cites)\n"
     "### Sources\n"
     "### Confidence"),
    ("human",
     "Partial answers from multiple chunks:\n{summaries}\n\nProduce the final answer.")
])



             # 1. chatbot qa yapisi


#get_answers, consolidates all the previous steps of the Retrieval Augmented Generation (RAG) pipeline into a single, cohesive unit.

#This function represents the complete workflow from receiving a user's question to generating a final, coherent answer.


def get_answers(query, k=4):
    from langchain_openai import ChatOpenAI
    from langchain_core.output_parsers import StrOutputParser
    from IPython.display import Markdown

    # 1) Retrieve top-k documents
    docs = retriever.invoke(query, k=k)

    # 2) Build context
    context = "\n\n".join(d.page_content for d in docs)

    # 3) Build a citations map the model can quote in output
    #    (ensures it actually has page numbers available)
    citations = "\n".join(
        f"[{i+1}] {d.metadata.get('source','unknown')} — p.{d.metadata.get('page','?')}"
        for i, d in enumerate(docs)
    )

    # 4) LLM
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, max_tokens=256)
    # max_tokens=1024)

    # 5) Chain
    chain = chat_template | llm | StrOutputParser()

    # 6) Invoke
    answer = chain.invoke({"question": query, "context": context, "citations": citations})

    # 7) Return (avoid adding another Sources block — model already includes it)
    return Markdown(answer)

In [61]:
our_query = "What are the two pre-training tasks used in BERT, and what does each one train the model to do?"

answer = get_answers(our_query, k=5)
answer

# Answer
The two pre-training tasks used in BERT are:

1. **Masked Language Model (MLM)**: This task involves masking a percentage of the input tokens and training the model to predict these masked tokens based on their context. It allows the model to learn bidirectional representations by considering both left and right context.

2. **Next Sentence Prediction (NSP)**: This task trains the model to predict whether a given sentence B follows sentence A in the original text. It helps the model understand the relationship between sentence pairs.

## Key Points (with page cites)
- BERT uses two unsupervised tasks for pre-training: Masked Language Model (MLM) and Next Sentence Prediction (NSP) (p.3).
- MLM trains the model to predict masked tokens in a sentence (p.3).
- NSP trains the model to determine if one sentence follows another (p.3).

### Sources
- [1] /content/drive/MyDrive/nlp/bert_article.pdf — p.3

### Confidence
High

In [62]:
our_query = "What ROUGE-L score does BERT report on CNN/DailyMail abstractive summarization using a pointer-generator"

answer = get_answers(our_query, k=5)
answer

I don't know.

In [64]:
our_query = "In the context of fine-tuning, what minimal number of parameters need to be randomly initialized when incorporating BERT with an extra output layer?"

answer = get_answers(our_query, k=5)
answer



# Answer
The minimal number of parameters that need to be randomly initialized when incorporating BERT with an extra output layer is very small, as the task-specific models are formed by adding only one additional output layer to the pre-trained model. 

## Key Points (with page cites)
- The task-specific models incorporate BERT with one additional output layer, requiring only a minimal number of parameters to be learned from scratch (page 4184).

### Sources
- [1] /content/drive/MyDrive/nlp/bert_article.pdf — p.8
- [2] /content/drive/MyDrive/nlp/bert_article.pdf — p.12
- [3] /content/drive/MyDrive/nlp/bert_article.pdf — p.13
- [4] /content/drive/MyDrive/nlp/bert_article.pdf — p.13
- [5] /content/drive/MyDrive/nlp/bert_article.pdf — p.2

### Confidence
High

In [50]:
# some question


# Relevant

# What are the two pre-training tasks used in BERT, and what does each one train the model to do?
# How do BERTBASE and BERTLARGE differ in L (layers), H (hidden size), A (attention heads), and total parameters?


# Irrelevant

# 3) What is the chemical formula of water?
# 4) What is the tallest mountain in South America?


#What ROUGE-L score does BERT report on CNN/DailyMail abstractive summarization using a pointer-generator





In [51]:
#                         works well  but may not be cost effective



from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


QUESTION_PROMPT = ChatPromptTemplate.from_template(
    """You are an data scientist assistant answering a user question using ONLY the provided context.

REQUIREMENTS
- If the context is insufficient, say: "I don't know based on the provided context."
- Be precise and neutral; do not speculate.
- Keep the main answer within 3–8 sentences unless explicitly asked for more.
- Cite support in brackets using metadata if available.

CONTEXT
{context}

QUESTION
{question}

ANSWER"""
)

COMBINE_PROMPT = ChatPromptTemplate.from_template(
    """You receive multiple candidate answers synthesized from different context chunks.

TASK
- Produce ONE final answer.
- Prefer statements directly supported by the context; drop anything weakly supported.
- If candidates conflict and evidence is unclear, state the uncertainty or contraversaries.
- Keep the main answer within 5–12 sentences.
- Consolidate citations in brackets if metadata is present.

CANDIDATE ANSWERS
{summaries}

FINAL ANSWER"""
)

# RetrievalQA icinde retriever en alakalı döküman parçalarını çeker,seçtiğin chain_type’a göre bu parçalar prompt’a yerleştirilir, LLM cevabı üretir

def get_answers(query, k=5):
    retriever = index.as_retriever(search_kwargs={"k": k})
    qa = RetrievalQA.from_chain_type(                        ####################
        llm=llm,
        retriever=retriever,
        chain_type="map_reduce",   # or "stuff" for simpler
        chain_type_kwargs={
            "question_prompt": QUESTION_PROMPT,
            "combine_prompt": COMBINE_PROMPT
        },
        return_source_documents=True,
    )
    out = qa.invoke({"query": query})   # in LC 0.2+, key is "query"
    return out['result']


In [52]:
our_query = " What are the two pre-training tasks used in BERT, and what does each one train the model to do?"

answer = get_answers(our_query, k=7)
answer

'The two pre-training tasks used in BERT are the Masked Language Model (MLM) and Next Sentence Prediction (NSP). The MLM involves masking a percentage of the input tokens and training the model to predict these masked tokens based on their context, allowing the model to learn deep bidirectional representations. The NSP task trains the model to determine if a given sentence logically follows another, which helps in understanding the relationship between sentences and is crucial for tasks requiring context comprehension across multiple sentences [Figure 1].'

In [53]:
Markdown(answer)

The two pre-training tasks used in BERT are the Masked Language Model (MLM) and Next Sentence Prediction (NSP). The MLM involves masking a percentage of the input tokens and training the model to predict these masked tokens based on their context, allowing the model to learn deep bidirectional representations. The NSP task trains the model to determine if a given sentence logically follows another, which helps in understanding the relationship between sentences and is crucial for tasks requiring context comprehension across multiple sentences [Figure 1].

In [54]:
our_query = "what is the relation between bert and tpu?"

answer = get_answers(our_query, k=5)
Markdown(answer)

Based on the provided context, there is insufficient information to formulate a definitive answer.

In [59]:


our_query = "In the context of fine-tuning, what minimal number of parameters need to be randomly initialized when incorporating BERT with an extra output layer?"

answer = get_answers(our_query, k=5)
Markdown(answer)





In the context of fine-tuning BERT with an additional output layer, the minimal number of parameters that need to be learned from scratch is the number of parameters in the new output layer itself. All other parameters of the pre-trained BERT model are typically retained and fine-tuned rather than initialized randomly. Therefore, only the parameters specific to the additional output layer would be initialized randomly.

In [56]:
def chat(k=5):
    from IPython.display import display
    while True:
        q = input("Q (blank/quit to exit): ").strip()
        if not q or q.lower() in {"q", "quit", "exit"}:
            break
        display(get_answers(q, k=k))
        print("-" * 80)

In [57]:
chat()

Q (blank/quit to exit): q
